# Fundamentals of Deep Learning

*Notebook 4.3: Predicting Housing Prices*

In [1]:
"""
The MIT License (MIT)
Copyright (c) 2021 NVIDIA
Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""


'\nThe MIT License (MIT)\nCopyright (c) 2021 NVIDIA\nPermission is hereby granted, free of charge, to any person obtaining a copy of\nthis software and associated documentation files (the "Software"), to deal in\nthe Software without restriction, including without limitation the rights to\nuse, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of\nthe Software, and to permit persons to whom the Software is furnished to do so,\nsubject to the following conditions:\nThe above copyright notice and this permission notice shall be included in all\ncopies or substantial portions of the Software.\nTHE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR\nIMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS\nFOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR\nCOPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER\nIN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OU

## Part I: Create the model

This example uses a neural network to solve a regression problem, using the Boston housing dataset.  The Boston Housing dataset is included in Keras, so it is simple to access using keras.datasets.boston_housing:

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Read and standardize the data.
boston_housing = keras.datasets.boston_housing
(raw_x_train, y_train), (raw_x_test,
    y_test) = boston_housing.load_data()

57026/57026 [==============================] - 0s 0us/step


We standardize both the training and test data by using the mean and standard deviation from the training data. The parameter axis=0 ensures that we compute the mean and standard deviation for each input variable separately. The resulting mean (and standard deviation) is a vector of means instead of a single value. That is, the standardized value of the nitric oxides concentration is not affected by the values of the per capita crime rate or any of the other variables.

In [3]:
x_mean = np.mean(raw_x_train, axis=0)
x_stddev = np.std(raw_x_train, axis=0)
x_train =(raw_x_train - x_mean) / x_stddev
x_test =(raw_x_test - x_mean) / x_stddev

Create the model by first instantiating the model object without any layers, and then add them one by one using the member method `add()`.
Next, complete the missing code in the below hidden layers using 64 ReLU neurons per layer.  Careful regarding the first layer as it needs to match the match the dataset. The output layer consists of a single neuron with a linear activation function.

In [4]:
# Create and train model.
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=[13]))
model.add(Dense(64, activation='relu')) # We are doing DL!
model.add(Dense(1, activation='linear'))

Use MSE as the loss function and use the `Adam` optimizer and compile method that we are interested in seeing the metric mean absolute error, and print out a summary of the model with `model.summary()` and then start training.  Experiment with different `batch sizes` and `epochs`.  Record the results for these experiments.

In [5]:
BATCH_SIZE = 16
EPOCHS = 500

model.compile(loss='mean_squared_error', optimizer='adam', metrics =['mean_absolute_error'])
model.summary()
history = model.fit(x_train, y_train, validation_data=(
    x_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE,
    verbose=2, shuffle=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                896       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5121 (20.00 KB)
Trainable params: 5121 (20.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500
26/26 - 1s - loss: 515.1454 - mean_absolute_error: 20.7593 - val_loss: 466.9162 - val_mean_absolute_error: 19.7256 - 1s/epoch - 48ms/step
Epoch 2/500
26/26 - 0s - loss: 355.6336 - mean_absolute_error: 16.6994 - val_loss: 262.3636 - val_mean_absolute_error: 14.2205 - 103ms/e

After the training is done, we use our model to predict the price for the entire test set.

In [6]:
predictions = model.predict(x_test)

4/4 [==============================] - 0s 3ms/step


Knowing the answers, let us check how far off you are:

In [7]:
expected =[7.939793, 18.455063, 20.115505, 32.14037]
for i in range(0, 4):
    assert ((predictions[i] - expected[i]) < 0.0001), "predicted value is too large"

AssertionError: predicted value is too large

## Part II: Tuning

Improve the above results by modifying the network to include more layers with more neurons.  Also, apply L1 and L2 regularization using different weight decay parameters: Start with $\lambda=0.1$ and try $\lambda=0.2$ and $\lambda=0.3$

In [8]:
from tensorflow.keras.layers import Dropout

# Create and train model.
model = Sequential()

### Add your layers here ###
model.add(Dense(128, activation='relu', input_shape=[13]))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam',
              metrics =['mean_absolute_error'])
model.summary()
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2, shuffle=True)

predictions = model.predict(x_test)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               1792      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                

Retry the above using dropout regularization.  What do you notice?

In [14]:
from tensorflow.keras.layers import Dropout

# Create and train model.
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=[13]))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics =['mean_absolute_error'])

model.summary()
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2, shuffle=True)

predictions = model.predict(x_test)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 128)               1792      
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 dense_20 (Dense)            (None, 128)               16512     
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_21 (Dense)            (None, 64)                8256      
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_22 (Dense)            (None, 1)                

In [15]:
for i in range(0, 4):
    print('Prediction: ', predictions[i, 0],
          ', true value: ', y_test[i])

Prediction:  9.184799 , true value:  7.2
Prediction:  18.46149 , true value:  18.8
Prediction:  19.800652 , true value:  19.0
Prediction:  30.083971 , true value:  27.0


Repeat the above by trying multiple parameter combinations:
- Using a combination of 64 to 128 neurons
- Using Dropout of 0.2 and 0.3
- Using L2 = 0.1 and 0.2

In [12]:
# Create and train model.
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=[13]))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics =['mean_absolute_error'])

model.summary()
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2, shuffle=True)

predictions = model.predict(x_test)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 128)               1792      
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_16 (Dense)            (None, 128)               16512     
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_17 (Dense)            (None, 64)                8256      
                                                                 
 dropout_8 (Dropout)         (None, 64)                0         
                                                                 
 dense_18 (Dense)            (None, 1)                

In [13]:
for i in range(0, 4):
    print('Prediction: ', predictions[i, 0],
          ', true value: ', y_test[i])

Prediction:  8.693324 , true value:  7.2
Prediction:  18.204363 , true value:  18.8
Prediction:  20.37779 , true value:  19.0
Prediction:  28.107452 , true value:  27.0


In [11]:
from tensorflow.keras.regularizers import l2

# Create and train model.
model = Sequential()
model.add(Dense(128, activation='relu',kernel_regularizer=l2(0.1),
    bias_regularizer=l2(0.1), input_shape=[13]))
#model.add(Dropout(0.3))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.2),
    bias_regularizer=l2(0.2)))
#model.add(Dropout(0.3))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.1),
    bias_regularizer=l2(0.1)))
#model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics =['mean_absolute_error'])

model.summary()
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2, shuffle=True)

predictions = model.predict(x_test)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 128)               1792      
                                                                 
 dense_12 (Dense)            (None, 128)               16512     
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 26625 (104.00 KB)
Trainable params: 26625 (104.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/500
26/26 - 1s - loss: 477.3607 - mean_absolute_error: 18.9017 - val_loss: 255.0473 - val_mean_absolute_error: 12.8703 - 1s/epoch - 47ms/ste

For each case, print first 4 predictions and record the answers!

In [10]:
for i in range(0, 4):
    print('Prediction: ', predictions[i, 0],
          ', true value: ', y_test[i])

Prediction:  8.441396 , true value:  7.2
Prediction:  18.564482 , true value:  18.8
Prediction:  19.98867 , true value:  19.0
Prediction:  26.604954 , true value:  27.0
